# OD Estimation Test
## IPF and MLE Estimates
#### Evan Faulkner
#### 2/25/2021

### Notation and algorithm described in:
#### https://dspace.mit.edu/handle/1721.1/37970

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

### Generate Random Single Line OD Problem

In [2]:
N_stops = 8 # number of stops on the bus line
i_max = 20 # max flow from i to j
T_ij = np.triu(np.random.randint(0,i_max+1,size=(N_stops,N_stops)),1) # true OD matrix, upper triangular, zero main diagonal
M_i = np.sum(T_ij,1) # boarding counts at stop i
M_j = np.sum(T_ij,0) # alighting counts at stop j
mask = T_ij>0 # mask off zero elements
t_ij = (np.sum(M_i)/((N_stops**2-N_stops)/2))*np.triu(np.ones((N_stops,N_stops)),1) # diffuse seed matrix
#t_ij = np.maximum(T_ij - np.triu(np.random.randint(0,math.ceil(0.7*i_max),\
#                    size=(N_stops,N_stops)),1),np.ones((N_stops,N_stops)))*mask

### IPF

In [3]:
eps = 1e-3 # stopping condition
delta = 1 # difference between the last 2 iterations
a_i = M_i/np.sum(t_ij,1) # a_i is the row factor, initialized with b_j^0=1
a_i[np.isnan(a_i)]=0
a_i[a_i<1] = 1
a_i = np.expand_dims(a_i,1)
b_j = M_j/np.sum(a_i*t_ij,0) # b_j is the column factor, initialized with a_i^0
b_j[np.isnan(b_j)]=0
b_j[b_j<1] = 1
b_j = np.expand_dims(b_j,1)

T_ipf = np.zeros((N_stops,N_stops))

while True:
    if delta>eps:
        a = M_i/np.sum(b_j[:,-1].T*t_ij,1)
        a[np.isnan(a)] = 0
        a[a<1] = 1
        a = np.expand_dims(a,1)
        b = M_j/np.sum(a*t_ij,0)
        b[np.isnan(b)] = 0
        b[b<1] = 1
        b = np.expand_dims(b,1)
        delta = np.abs(np.mean(a-a_i[:,-1])+np.mean(b-b_j[:,-1]))
        #print('delta = ',delta)
        a_i = np.c_[a_i,a]
        b_j = np.c_[b_j,b]
    else:
        print(f'a_i = {np.round(a_i[:,-1], 2)}', f'\nb_j = {np.round(b_j[:,-1], 2)}')
        break
    
for i in range(N_stops):
    for j in range(N_stops):
        T_ipf[i,j] = a_i[i,-1]*b[j,-1]*t_ij[i,j] # compute the estimated OD matrix from the final a_i, b_j, and seed entries

a_i = [1.   1.   1.28 1.14 1.07 1.   1.17 1.  ] 
b_j = [1.   1.35 1.   1.02 1.   1.2  1.   1.  ]


<ipython-input-3-687f0b6d8783>:3: RuntimeWarning: invalid value encountered in true_divide
  a_i = M_i/np.sum(t_ij,1) # a_i is the row factor, initialized with b_j^0=1
<ipython-input-3-687f0b6d8783>:7: RuntimeWarning: invalid value encountered in true_divide
  b_j = M_j/np.sum(a_i*t_ij,0) # b_j is the column factor, initialized with a_i^0
<ipython-input-3-687f0b6d8783>:16: RuntimeWarning: invalid value encountered in true_divide
  a = M_i/np.sum(b_j[:,-1].T*t_ij,1)
<ipython-input-3-687f0b6d8783>:20: RuntimeWarning: invalid value encountered in true_divide
  b = M_j/np.sum(a*t_ij,0)


### Intervening Opportunity Method

### Comparison

In [4]:
print(f'Elementwise Error in T_ipf:\n {np.round((T_ij-T_ipf),2)}')
print(f'Error in T_ipf: {np.round(np.sum(np.abs(T_ij-T_ipf))/np.sum(np.abs(T_ij)),2)}')

Elementwise Error in T_ipf:
 [[  0.     0.    -7.11  -2.29 -11.11   1.69   0.89  -7.11]
 [  0.     0.    -7.11   0.71   2.89  -5.31  -6.11   3.89]
 [  0.     0.     0.     1.57  -0.2    0.99  -1.2   -1.2 ]
 [  0.     0.     0.     0.    -0.63  -3.13   2.37   1.37]
 [  0.     0.     0.     0.     0.     5.76  -9.89   4.11]
 [  0.     0.     0.     0.     0.     0.    -5.11  -1.11]
 [  0.     0.     0.     0.     0.     0.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.     0.     0.  ]]
Error in T_ipf: 0.3
